In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import os
import optuna
import logging
import xgboost as xgb
import sys
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

"""## Variables"""

current_storage = os.path.join("d:", "data")
vn30f_storage = os.path.join(current_storage, 'vn30f')

start_time = 0
now_time = 9999999999
symbol = 'VN30F1M'
rolling_window = 1 # Số phút muốn dự đoán tiếp theo

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "btc1-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

"""## Formulas"""

def choose_position(roi, trade_threshold = 0.0001):
    pos =0
    # Predict position base on change in future
    if roi > trade_threshold:
        pos = 1
    elif roi < -trade_threshold:
        pos = -1
    else:
        pos = 0

    return pos

def backtest_position_ps(position, price, percentage, periods):
    # Shift positions to align with future price changes and handle NaN by filling with 0
    pos = pd.Series(position, index=pd.Series(price).index).shift(1).fillna(0)
    pos = pd.Series(pos).rolling(periods).sum() #pos for 10 hour predict

    price_array = pd.Series(price).shift(1).fillna(0)

    pos_diff = pos.diff()
    fee = pos_diff*price_array*0.05*0.01

    # Calculate price changes over the given periods
    ch = pd.Series(price) - price_array

    # Calculate total PnL
    total_pnl = pos*ch - fee
    return total_pnl

def calculate_sharpe_ratio(pnl):
    pnl = np.diff(pnl)
    std = np.std(pnl) if np.std(pnl) != 0 else 0.001
    sharpe = np.mean(pnl)/std*np.sqrt(252)
    return sharpe

def sharpe_for_vn30f(y_pred, y_price, trade_threshold, fee_perc, periods):

    # Predict position base on change in future
    pos = [choose_position(roi, trade_threshold) for roi in y_pred]
    pos = np.array(pos)

    # Calculate PNL
    pnl = backtest_position_ps(pos, y_price, fee_perc, periods)
    pnl = np.cumsum(pnl)

    # Standardalize PNL to date
    daily_pnl = [pnl.iloc[i] for i in range(0, len(pnl), 241)]
    daily_pnl = pd.Series(daily_pnl).fillna(0)

    # Calculate Sharpe
    sharpe = calculate_sharpe_ratio(daily_pnl)

    return pos, pnl, daily_pnl, sharpe

def calculate_hitrate(pos_predict, pos_true):
    if len(pos_predict) != len(pos_true):
        raise ValueError("Độ dài của hai mảng không khớp")

    # Tính số lượng dự đoán đúng (các phần tử tương ứng giống nhau)
    correct_predictions = np.sum(pos_predict == pos_true)

    # Tính tỷ lệ hit rate
    hit_rate_value = correct_predictions / len(pos_predict)

    return hit_rate_value

"""# Function for data"""

def scale_data(data):
    scaler = StandardScaler()
    data = np.where(np.isinf(data), np.nan, data)
    data = pd.DataFrame(data)
    data = data.fillna(0)
    scaler.fit(data)
    data=pd.DataFrame(scaler.transform(data), index=data.index, columns=data.columns)

    return data

def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

def split_optuna_data(data):
    """
    Hàm này chia dữ liệu thành các tập train và test để sử dụng trong quá trình tối ưu hóa bằng Optuna.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu train (đã được chuẩn hóa).
    pandas.DataFrame: DataFrame chứa dữ liệu test (đã được chuẩn hóa).
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu train.
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu test.
    """
    # Chia dữ liệu thành tập train và tập hold out
    train_data, _ = split_data(data)

    # Loại bỏ các cột không cần thiết
    if 'Open' in train_data.columns:
        optuna_data = train_data.drop(['Open','High','Low','Close','Volume', 'Return', 'Date', 'time'], axis=1)
    else:
        optuna_data = train_data.drop(['Close', 'Return', 'Date', 'time'], axis=1)

    # Chuẩn hóa dữ liệu
    optuna_data = scale_data(optuna_data)

    # Chia dữ liệu thành tập train và tập test
    X_train, X_valid, y_train, y_valid = train_test_split(optuna_data, train_data['Return'], test_size=0.5, shuffle=False)

    return X_train, X_valid, y_train, y_valid, train_data

data = pd.read_csv('/kaggle/input/tempdataa/save_data.csv')

def objective(trial, X_train, X_valid, y_train, y_valid, y_price):

    # Select features based on Optuna's suggestions
    selected_features = []

    at_least_one_feature = False

    for col in X_train.columns:
        select_feature = trial.suggest_categorical(col, [0, 1])
        if select_feature:
            selected_features.append(col)
            at_least_one_feature = True

    # If no feature was selected, force selection of at least one feature
    if not at_least_one_feature:
        # Randomly select one feature to be included
        forced_feature = trial.suggest_categorical('forced_feature', X_train.columns.tolist())
        selected_features.append(forced_feature)

    for t in trial.study.trials:
        if t.state != optuna.trial.TrialState.COMPLETE:
            continue
        if t.params == trial.params:
            return np.nan # t.values  # Return the previous value without re-evaluating i

    trade_threshold  = 0.0001

    # Use only the selected features in training
    X_train_selected = X_train[selected_features]
    X_valid_selected = X_valid[selected_features]

    # Train the model
    model = GradientBoostingRegressor()
    model.fit(X_train_selected, y_train)

    y_pred_train = model.predict(X_train_selected)
    y_pred_valid = model.predict(X_valid_selected)

    _, pos_is, _, sharpe_is = sharpe_for_vn30f(y_pred_train, y_price[:len(y_pred_train)], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)
    _, pos_os, _, sharpe_oos = sharpe_for_vn30f(y_pred_valid, y_price[len(y_pred_train):], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)

    print('Trade times in sample:',len(pos_is[pos_is != 0]), 'Trade times out sample:',len(pos_os[pos_os != 0]))

    return sharpe_oos, abs((abs(sharpe_is / sharpe_oos))-1)

X_train, X_valid, y_train, y_valid, train_data = split_optuna_data(data)
"""## Define number of trials (no 2)"""

# Create a study object and optimize the objective function
study = optuna.create_study(directions=['maximize', 'minimize'])
unique_trials = 200

while unique_trials > len(set(str(t.params) for t in study.trials)):
    study.optimize(lambda trial: objective(trial, X_train, X_valid, y_train, y_valid, train_data['Close']), n_trials=1)
    study.trials_dataframe().fillna(0).sort_values('values_0').to_csv('xgb_feature_trials.csv')
    joblib.dump(study, 'xgbmodel.pkl')

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
[I 2024-06-29 15:07:01,516] A new study created in memory with name: no-name-d14ddfb9-ea1f-4b4f-8e6c-26b913399dc8


A new study created in memory with name: no-name-d14ddfb9-ea1f-4b4f-8e6c-26b913399dc8


[I 2024-06-29 15:07:33,165] Trial 0 finished with values: [-0.03514148324426026, 105.65608740498914] and parameters: {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 0}. 


Trade times in sample: 118243 Trade times out sample: 118188
Trial 0 finished with values: [-0.03514148324426026, 105.65608740498914] and parameters: {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 0}. 


[I 2024-06-29 15:07:51,144] Trial 1 finished with values: [0.5142120105772001, 5.723334027140776] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 0, 13: 0, 14: 1}. 


...

Trial 231 failed with value nan.


[I 2024-06-29 16:33:01,862] Trial 232 finished with values: [0.002630181323276148, 973.278741188027] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0}. 


Trade times in sample: 117124 Trade times out sample: 115181
Trial 232 finished with values: [0.002630181323276148, 973.278741188027] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0}. 


[I 2024-06-29 16:33:29,919] Trial 233 finished with values: [0.8777481689209049, 2.825338242095819] and parameters: {0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 1, 12: 1, 13: 1, 14: 0}. 


Trade times in sample: 117530 Trade times out sample: 118189
Trial 233 finished with values: [0.8777481689209049, 2.825338242095819] and parameters: {0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 1, 12: 1, 13: 1, 14: 0}. 


In [2]:
study.trials_dataframe().fillna(0).sort_values('values_0')

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_0,params_1,params_2,params_3,...,params_7,params_8,params_9,params_10,params_11,params_12,params_13,params_14,system_attrs_nsga2:generation,state
30,30,-2.897249,0.128707,2024-06-29 15:20:33.906136,2024-06-29 15:20:59.061926,0 days 00:00:25.155790,1,0,1,1,...,0,0,0,0,0,1,0,0,0,COMPLETE
60,60,-2.713530,0.174448,2024-06-29 15:33:49.389698,2024-06-29 15:34:20.861950,0 days 00:00:31.472252,1,0,1,1,...,0,0,1,0,0,0,1,0,1,COMPLETE
178,178,-2.619762,0.449077,2024-06-29 16:18:19.993437,2024-06-29 16:18:34.437038,0 days 00:00:14.443601,0,0,1,0,...,0,0,0,0,1,1,0,0,3,COMPLETE
96,96,-2.595069,0.119420,2024-06-29 15:47:46.879618,2024-06-29 15:48:11.708562,0 days 00:00:24.828944,0,1,1,0,...,0,0,1,1,1,1,0,0,1,COMPLETE
176,176,-2.488940,0.114713,2024-06-29 16:17:23.757119,2024-06-29 16:17:55.100963,0 days 00:00:31.343844,0,1,1,0,...,0,0,1,0,1,1,1,0,3,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,121,2.913434,0.840873,2024-06-29 15:56:24.108335,2024-06-29 15:56:40.074264,0 days 00:00:15.965929,1,0,0,0,...,0,0,0,0,0,0,1,1,2,COMPLETE
207,207,2.958789,0.495511,2024-06-29 16:26:06.951756,2024-06-29 16:26:33.641513,0 days 00:00:26.689757,0,0,0,0,...,1,1,0,1,1,1,1,1,3,COMPLETE
141,141,2.994933,0.477463,2024-06-29 16:05:07.538896,2024-06-29 16:05:33.785531,0 days 00:00:26.246635,0,0,0,0,...,1,1,0,1,0,1,1,1,2,COMPLETE
104,104,3.032966,0.665379,2024-06-29 15:50:21.883650,2024-06-29 15:50:37.910711,0 days 00:00:16.027061,1,0,0,0,...,0,0,0,0,1,0,1,1,1,COMPLETE
